In [ ]:
%pip install Mistral
%pip install langchain
%pip install kagglehub
%pip install reportlab
%pip install langchain_community
%pip install --upgrade mistralai
%pip install pypdf
%pip install sentence-transformers
%pip install -U langchain-huggingface
%pip install -U transformers
%pip install faiss-gpu
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os.environ["MISTRAL_API_KEY"]=""

In [3]:
import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
#from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
#from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
import logging
import re
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from mistralai import Mistral
from langchain.llms.base import LLM
from typing import Optional, List, Dict, Any
from pydantic import Field, PrivateAttr


/Users/sulabhadhikari/Desktop/machine-learning/project1/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logging.getLogger("httpx").setLevel(logging.WARNING)

In [5]:
def clean_text(text):
    """Clean and normalize text for embedding"""
    if not isinstance(text, str):
        return ""
    
    # Remove non-printable characters
    text = re.sub(r'[^\x20-\x7E\u00A0-\u024F\u1E00-\u1EFF]', ' ', text)
    
    # Normalize whitespace
    text = ' '.join(text.split())
    
    # Remove any remaining control characters
    text = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)
    
    return text.strip()

In [6]:


def convert_pdf(file, output_folder="books"):
    os.makedirs(output_folder, exist_ok=True)
    styles = getSampleStyleSheet()

    for txt_file in glob.glob(os.path.join(file,"*.txt")):
        with open(txt_file, "r", encoding="utf-8") as f:
            text = f.read()
        pdf_file = os.path.join(output_folder, os.path.basename(txt_file).replace(".txt", ".pdf"))
        doc = SimpleDocTemplate(pdf_file)
        content = [Paragraph(text, styles["Normal"])]
        doc.build(content)
    print("All text file converted to pdf")

convert_pdf("./books")



All text file converted to pdf


In [7]:


class MistralLangChainLLM(LLM):

    model_name:str = Field(default="mistral-tiny")
    _client:Any = PrivateAttr()
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._client = Mistral(api_key=os.environ["MISTRAL_API_KEY"])
       

    @property
    def _llm_type(self)->str:
        return "mistral"
    
    def _call(self, prompt:str, stop: Optional[List[str]] = None, **kwargs:Any) ->str:
        try:
            response = self._client.chat.complete(
                model=self.model_name,
                messages=[{"role":"user", "content":prompt}]
            )
            return response.choices[0].message.content
        except Exception as e:
             logger.error(f"Error calling Mistral API: {e}")
             return "I encountered an error while processing your request."
    @property
    def _identifying_params(self) -> Dict[str, Any]:
        return {"model_name": self.model_name}

In [ ]:
#load pdfs
pdf_folder = "books"
all_docs=[]
for pdf in glob.glob(f"{pdf_folder}/*.pdf"):
    try:
        logger.info(f"Loading PDF: {pdf}")
        loader = PyPDFLoader(pdf)
        documents = loader.load()
        for i, doc in enumerate(documents):
            if hasattr(doc, 'page_content') and doc.page_content and isinstance(doc.page_content, str):
                
                cleaned_content = ' '.join(doc.page_content.split())
                if len(cleaned_content) > 50:  # Minimum length
                    # Create a new document with only the cleaned content
                    new_doc = Document(
                        page_content=cleaned_content,
                        metadata={
                            **doc.metadata,
                            'source_file': os.path.basename(pdf),
                            'chunk_id': f"{os.path.basename(pdf)}_page_{i}"
                        }
                    )
                    all_docs.append(new_doc)
    except Exception as e:
        logger.error(f"Error loading Pdf{pdf}: {e}")

logger.info(f"Loaded {len(all_docs)} document after cleaning")

INFO:__main__:Loading PDF: books/bitcoin.pdf
INFO:__main__:Loading PDF: books/altcoin.pdf
INFO:__main__:Loading PDF: books/1713ccb6-a785-4529-ba09-27f1cdc2e014.pdf
INFO:__main__:Loading PDF: books/Web_3_0_The_New_Era_of_Data.pdf
INFO:__main__:Loading PDF: books/file.pdf
INFO:__main__:Loading PDF: books/ethereum.pdf
INFO:__main__:Loading PDF: books/dfc41b13-a530-458f-ac5e-e610336d911a.pdf
INFO:__main__:Loading PDF: books/146d5bf0-453b-493e-8167-b59a229e676d.pdf
INFO:__main__:Loading PDF: books/88a17195-ff85-4066-98d0-da53e7834758.pdf
INFO:__main__:Loading PDF: books/blockchain.pdf
INFO:__main__:Loading PDF: books/3608d9cb-23be-4332-abf8-edb54b75be98.pdf
INFO:__main__:Loading PDF: books/14a9eb3a-f6f6-4824-b5ef-f8923fd4a173.pdf
INFO:__main__:Loading PDF: books/Ethereum_Introduction_Expectation_and_Implementati.pdf
INFO:__main__:Loaded 1906 document after cleaning


In [9]:
# split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150, separators=["\n\n", "\n", " ", ""], length_function=len)
chunks = splitter.split_documents(all_docs)
logger.info(f"Created {len(chunks)} chunks after splitting")

INFO:__main__:Created 5351 chunks after splitting


In [10]:

# create vector store
## Filter and clean chunks
valid_chunks = []
for i, chunk in enumerate(chunks):
    if(hasattr(chunk, 'page_content') and chunk.page_content and isinstance(chunk.page_content, str)):
        clean_content = clean_text(chunk.page_content)
        
        if 50<=len(cleaned_content)<=2000:
            valid_chunk = Document(
                page_content=cleaned_content,
                metadata={
                    **chunk.metadata,
                    'original_length': len(chunk.page_content),
                    'cleaned_length': len(cleaned_content)
                }
            )
            valid_chunks.append(valid_chunk)
        else:
            logger.warning(f"Chunk {i} length out of bounds: {len(cleaned_content)}")



logger.info(f"Final valid chunks: {len(valid_chunks)}")

INFO:__main__:Final valid chunks: 5351


In [ ]:
try:

     # First, test the embedding model with a simple string
    test_texts = ["This is a test sentence for embedding validation."]

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': 'cpu'},  # Specify device
            encode_kwargs={'normalize_embeddings': True, "batch_size":32})
    
    # Test embedding
    test_embeddings = embeddings.embed_documents(test_texts)
    logger.info("Embedding test successful")

    # Extract texts and validate
    texts = []
    metadatas = []

    for i, chunk in enumerate(valid_chunks):
        text = chunk.page_content
        if isinstance(text, str) and text.strip():
            texts.append(text)
            metadatas.append(chunk.metadata)
        else:
            logger.warning(f"Skipping invalid text at index {i}")

    logger.info(f"Preparing to embed {len(texts)} valid texts")

    # Create FAISS index in smaller batches if needed
    if len(texts) > 1000:
        logger.info("Large dataset detected, embedding in smaller batches...")
       
    
    vectorestore = FAISS.from_texts(
        texts=texts,
        embedding=embeddings,
        metadatas=metadatas
    )
    
    vectorestore.save_local("crypto_index")
    logger.info("Vector store created and saved successfully")
except Exception as e:
    logger.error(f"Error creating vector store: {e}")
    
    # Detailed debugging
    logger.info("Debugging first 10 texts:")
    for i, text in enumerate(texts[:10]):
        logger.info(f"Text {i}: Type={type(text)}, Length={len(text)}")
        logger.info(f"Content preview: {text[:100]}...")
        logger.info(f"Contains non-ASCII: {any(ord(c) > 127 for c in text)}")
    
    # Try to identify problematic texts
    problematic_indices = []
    for i, text in enumerate(texts):
        if not isinstance(text, str) or not text.strip():
            problematic_indices.append(i)
        elif any(ord(c) < 32 and c not in '\n\r\t' for c in text):
            problematic_indices.append(i)
    
    if problematic_indices:
        logger.error(f"Problematic texts at indices: {problematic_indices[:10]}")
    
    raise


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:__main__:Embedding test successful
INFO:__main__:Preparing to embed 5351 valid texts
INFO:__main__:Large dataset detected, embedding in smaller batches...
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.
INFO:__main__:Vector store created and saved successfully


In [12]:
#Build retriever
retriever = vectorestore.as_retriever(search_kwargs={"k":5})

#define Mistral LLM(replace with actual Mistral Integration)
llm = MistralLangChainLLM(model_name="mistral-tiny")

#prompt
prompt_template = """
You are an expert on cryptocurrency and blockchain. Answer only using the context below.
If the answer is not in the context, reply:"I don't know."

Please provide well-structured answers with proper paragraphs and formatting.

Context: {context}
Question: {question}
Answer:
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)

# RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt":prompt},
    return_source_documents=False
)

In [17]:
# Test function
def ask_question(question):
    try:
        result = qa.invoke({"query": question})
        #print(f"Question: {question}")
        print(result['result'])
        # if 'source_documents' in result:
        #     print("Sources:")
            # for i, doc in enumerate(result['source_documents'][:3]):
            #     print(f"  {i+1}. {doc.page_content[:100]}...")
        #print("-" * 50)
    except Exception as e:
        print(f"Error asking question: {e}")

question = input("Enter the question you want to ask chatbot\n")
ask_question(question)

I don't know. The provided context does not mention anything named "abcde".
